# Viable $f_1, f_2$ pairs

Let's check many possible candidates for our frequency tags based on which bands of activity they fall in to, and where the possible intermodulation responses could fall for many common IM components.

We'll start by defining functions for finding possible tag frequencies ($f_1$ and $f_2$) for different refresh rates, and determining those possible IM components (also harmonics). 

We will also define the frequency bands:
 - $\delta$: 0-4 Hz
 - $\theta$: 4-8 Hz
 - $\alpha$: 8-12 Hz
 - $\beta$: 12-30 Hz
 - $\gamma$: 30-45 Hz
 - $f_{\text{line}}$: 50 Hz line noise
 - $f_{\text{choo-choo}}$: 15-17 Hz (50Hz / 3 from 3-phase train pantograph)


In [109]:
import numpy as np
import pandas as pd
import itertools as it

mults = (1, 2, -1, -2)
mpairs = list(it.product(mults, mults))


def get_possible_tags(refresh, min_freq=1.0):
    baseperiod = 2 / refresh
    maxmult = min_freq / baseperiod
    mults = np.arange(1, maxmult + 1)
    return 2 / (baseperiod * mults)


def get_possible_IM(frow):
    f1, f2 = frow.iloc[0], frow.iloc[1]
    m1, m2 = np.meshgrid(mults, mults)

    def multstr(m, pos):
        match (m, pos):
            case (1, 1):
                return ""
            case (-1, 1):
                return "-"
            case (1, 2):
                return "+"
            case (_, 1):
                return str(m)
            case (_, 2):
                return f"+{m}"

    outdf = pd.DataFrame(
        np.ones((16, 1)).T * np.nan,
        columns=[f"{multstr(m1, 1)}f1{multstr(m2, 2)}f2" for m1, m2 in mpairs],
    )
    for i in range(4):
        for j in range(4):
            res = f1 * m1[i, j] + f2 * m2[i, j]
            if res >= 0:
                outdf.iloc[0, i * 4 + j] = res
            else:
                continue
    return outdf


def get_rate_band(f, bands):
    for name in bands:
        band = bands[name]
        if band["bounds"][0] <= f < band["bounds"][1]:
            return band["name"]
    return "No established"


bands = {
    "delta": {"bounds": (0, 4), "name": "Delta (0-4 Hz)"},
    "theta": {"bounds": (4, 8), "name": "Theta (4-8 Hz)"},
    "alpha": {"bounds": (8, 12), "name": "Alpha (8-12 Hz)"},
    "beta": {"bounds": (12, 30), "name": "Beta (12-30 Hz)"},
    "gamma": {"bounds": (30, 45), "name": "Gamma (30-45 Hz)"},
    "choochoo": {"bounds": (15, 17), "name": "Choo Choo (15-17 Hz, ~50Hz / 3)"},
    "line": {"bounds": (49, 51), "name": "Line noise (50 Hz)"},
}

Next lets define the desired refresh rate (change as you please) and look at the outputs. We will generate all possible pairs at this refresh rate, and all positive IM bands.

In [110]:
REFRESH = 240
rates = pd.DataFrame(get_possible_tags(REFRESH), columns=["rate"])
rates["band"] = rates["rate"].apply(lambda x: get_rate_band(x, bands))
ratepairs = pd.merge(rates, rates, how="cross", suffixes=("_1", "_2"))
imvals = [get_possible_IM(ratepairs.loc[i, ["rate_1", "rate_2"]]) for i in range(len(ratepairs))]
imvals = pd.concat(imvals, axis=0, ignore_index=True)
pairsIM = pd.concat((ratepairs, imvals), axis=1)
pairsIM = pairsIM[pairsIM["rate_1"] < pairsIM["rate_2"]]
print(f"{len(pairsIM)} pairs of rates initially possible")

7140 pairs of rates initially possible


We will naturally want to filter out certain bands that we definitely don't want to see either the main tags $F_1, F_2$ or the intermodulation tags appear in.

In our case we will exclude $\delta$, $\theta$, and $\alpha$ bands, as well as the line and train frequencies $f_{\text{line}}, f_{\text{choo-choo}}$. 

We will filter out all pairs where at least one tag is in any of these bands, and those where all of the IM frequencies are in these bands.

In [111]:
BADBANDS = ["delta", "theta", "alpha", "choochoo", "line"]
UPPERBOUND = 30  # Upper bound of ideal IM freqs
IM_UPPER = 60


def bandmask(f, band):
    bounds = bands[band]["bounds"]
    return (bounds[0] <= f) & (f < bounds[1])


def multibandmask(f, bands, ub=UPPERBOUND):
    retmask = np.ones_like(f, dtype=bool)
    for band in bands:
        retmask &= ~bandmask(f, band)
    if ub is not None:
        retmask &= f < UPPERBOUND
    return retmask


pairfilter = pairsIM[
    multibandmask(pairsIM["rate_1"], BADBANDS) & multibandmask(pairsIM["rate_2"], BADBANDS)
]
print(f"{len(pairfilter)} pairs remain after primary band filter")


immasks = np.zeros((len(pairfilter), int(len(mults) ** 2)), dtype=bool)
for col in range(len(mults) ** 2):
    immasks[:, col] = multibandmask(pairfilter.iloc[:, -col], BADBANDS, ub=IM_UPPER)
immask = np.any(immasks, axis=1)
print(f"{np.sum(immask)} pairs remain after IM band filter")
goodIMs = pairfilter[immask].copy()
colmasks = [(col, mask) for col, mask in zip(goodIMs.columns[-16:], immasks.T)]
for col, mask in colmasks:
    goodIMs.loc[:, col] = goodIMs[col].where(mask, np.nan)

# Drop IMs that have no valid values
goodIMs.dropna(axis=1, how="all", inplace=True)

45 pairs remain after primary band filter
45 pairs remain after IM band filter


## Final pairs

This leaves us with 45 candidate pairs of frequencies that we could use for the frequency tags:

In [115]:
pd.set_option("display.precision", 2)
goodIMs.sort_values("rate_1")


,rate_1,band_1,rate_2,band_2,f1+f2,2f1+-2f2,-f1+2f2
2298,12.00,Beta (12-30 Hz),12.63,Beta (12-30 Hz),24.63,NaN,NaN
2288,12.00,Beta (12-30 Hz),26.67,Beta (12-30 Hz),38.67,NaN,NaN
2289,12.00,Beta (12-30 Hz),24.00,Beta (12-30 Hz),36.00,NaN,0.00
2290,12.00,Beta (12-30 Hz),21.82,Beta (12-30 Hz),33.82,NaN,2.18
2297,12.00,Beta (12-30 Hz),13.33,Beta (12-30 Hz),25.33,NaN,NaN
2292,12.00,Beta (12-30 Hz),18.46,Beta (12-30 Hz),30.46,NaN,5.54
2293,12.00,Beta (12-30 Hz),17.14,Beta (12-30 Hz),29.14,NaN,NaN
2296,12.00,Beta (12-30 Hz),14.12,Beta (12-30 Hz),26.12,NaN,NaN
2291,12.00,Beta (12-30 Hz),20.00,Beta (12-30 Hz),32.00,NaN,NaN
2168,12.63,Beta (12-30 Hz),26.67,Beta (12-30 Hz),39.30,NaN,NaN
